In [20]:
trips = pq.read_table("data/trips/green_tripdata_2019-01.parquet")
trips: pd.DataFrame = trips.to_pandas()
print(trips)

        VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
0              2  2018-12-21 15:17:29   2018-12-21 15:18:57   
1              2  2019-01-01 00:10:16   2019-01-01 00:16:32   
2              2  2019-01-01 00:27:11   2019-01-01 00:31:38   
3              2  2019-01-01 00:46:20   2019-01-01 01:04:54   
4              2  2019-01-01 00:19:06   2019-01-01 00:39:43   
...          ...                  ...                   ...   
672100         2  2019-01-31 23:08:00   2019-01-31 23:28:00   
672101         2  2019-01-31 23:28:00   2019-01-31 23:47:00   
672102         2  2019-01-31 23:14:00   2019-01-31 23:29:00   
672103         2  2019-01-31 23:41:00   2019-01-31 23:48:00   
672104         2  2019-01-31 23:16:00   2019-01-31 23:39:00   

       store_and_fwd_flag  RatecodeID  PULocationID  DOLocationID  \
0                       N         1.0           264           264   
1                       N         1.0            97            49   
2                       N         1.

# Clean data

In [22]:
import pyarrow.parquet as pq
import pandas as pd

def readAndCleanData(filename, yellow=True):
    pickupTime = "tpep_pickup_datetime" if yellow else "lpep_pickup_datetime"
    dropoffTime = "tpep_dropoff_datetime" if yellow else "lpep_dropoff_datetime"

    __trips = pq.read_table(filename)
    __trips: pd.DataFrame = __trips.to_pandas()
    __trips = __trips[[pickupTime, dropoffTime, "trip_distance", "PULocationID", "DOLocationID"]]

    # rename pickupTime and dropoffTime to "pickup_time" and "dropoff_time" respectively
    __trips = __trips.rename(columns={pickupTime: "pickup_time", dropoffTime: "dropoff_time"})

    # remove all rows with trip distance of 0
    __trips = __trips[__trips.trip_distance > 0.0]

    # remove all rows with pickup time that is not in year 2019
    __trips = __trips[__trips.pickup_time.dt.year == 2019]

    # remove all rows with pickup time after dropoff time
    __trips = __trips[__trips["pickup_time"] <= __trips["dropoff_time"]]

    # add column with trip time in seconds
    __trips["trip_time"] = (pd.to_datetime(__trips["dropoff_time"]) - pd.to_datetime(__trips["pickup_time"])).dt.total_seconds()

    # remove dropoff time column
    __trips = __trips.drop(columns=["dropoff_time"])

    # change pickup_time year to start of epoch year
    __trips["pickup_time"] = __trips["pickup_time"].apply(lambda x: x.replace(year=1970))

    # convert pickupTime to numeric values
    __trips["pickup_time"] = pd.to_numeric(__trips["pickup_time"])

    return __trips

trips = readAndCleanData("data/trips/yellow_tripdata_2019-08.parquet")
print(f"Yellow trip data: \n{trips}\n")
# trips = readAndCleanData("data/trips/green_tripdata_2019-01.parquet", False)
# print(f"Green trip data: \n{trips}\n")

# plot pickupTime trip_distance, PUlocationID, and DOLocationID] vs trip_time
# trips.plot(x=["pickupTime", "trip_distance", "PULocationID", "DOLocationID"], y="trip_time", kind="scatter", figsize=(12, 6))
# plt.scatter(trips["pickupTime"], trips["trip_distance"], trips["PULocationID"], trips["DOLocationID"], c=trips["trip_time"], cmap="tab20")


Yellow trip data: 
               pickup_time  trip_distance  PULocationID  DOLocationID  \
2        18319490000000000           7.10           145           145   
5        18319640000000000           7.30           234            61   
9        18317785000000000           6.50            90             7   
10       18319428000000000           2.10           164            79   
11       18317267000000000           0.62            13            13   
...                    ...            ...           ...           ...   
6073352  20991600000000000          14.05           142            89   
6073353  20994900000000000           6.83           232            74   
6073354  20993040000000000          19.88           216           133   
6073355  20992980000000000          15.09           143           215   
6073356  20992740000000000           4.07           159           235   

         trip_time  
2            102.0  
5           2065.0  
9           1914.0  
10           685.0  

In [27]:
import os

# read all parquet files into one pandas dataframe
trips = pd.DataFrame()
directory = "data/trips"
for file in os.listdir(directory):
    if file.endswith(".parquet"):
        print(f"Reading {file}")
        trips = pd.concat([trips, readAndCleanData(f"{directory}/{file}", file.startswith("yellow"))])

# save all trips into one parquet
print(f"Saving all trips into one parquet")
trips.to_parquet("data/trips/all_trips.parquet")
print(f"Saved")

Reading all_trips.parquet
Reading green_tripdata_2019-01.parquet
Reading green_tripdata_2019-02.parquet
Reading green_tripdata_2019-03.parquet
Reading green_tripdata_2019-04.parquet
Reading green_tripdata_2019-05.parquet
Reading green_tripdata_2019-06.parquet
Reading green_tripdata_2019-07.parquet
Reading green_tripdata_2019-08.parquet
Reading green_tripdata_2019-09.parquet
Reading green_tripdata_2019-10.parquet
Reading green_tripdata_2019-11.parquet
Reading green_tripdata_2019-12.parquet
Reading yellow_tripdata_2019-01.parquet
Reading yellow_tripdata_2019-02.parquet
Reading yellow_tripdata_2019-03.parquet
Reading yellow_tripdata_2019-04.parquet
Reading yellow_tripdata_2019-05.parquet
Reading yellow_tripdata_2019-06.parquet
Reading yellow_tripdata_2019-07.parquet
Reading yellow_tripdata_2019-08.parquet
Reading yellow_tripdata_2019-09.parquet
Reading yellow_tripdata_2019-10.parquet
Reading yellow_tripdata_2019-11.parquet
Reading yellow_tripdata_2019-12.parquet
Saving all trips into one 

# Prepare training of the model

## Read all data from combined parquet file

In [2]:
import pandas as pd

# Read all_trips.parquet
trips = pd.read_parquet("data/trips/all_trips.parquet")

# get ["pickupTime", "trip_distance", "PULocationID", "DOLocationID"] from trips as X values
x = trips[["pickup_time", "trip_distance", "PULocationID", "DOLocationID"]].to_numpy()
print(f"X values: \n{x}\n")

# get ["trip_time"] from trips as y values
y = trips[["trip_time"]].to_numpy().ravel()
print(f"Y values: \n{y}\n")

X values: 
[[6.1600000e+11 8.6000000e-01 9.7000000e+01 4.9000000e+01]
 [1.6310000e+12 6.6000000e-01 4.9000000e+01 1.8900000e+02]
 [2.7800000e+12 2.6800000e+00 1.8900000e+02 1.7000000e+01]
 ...
 [3.1533113e+16 3.7500000e+00 1.4800000e+02 2.4600000e+02]
 [3.1535841e+16 6.4600000e+00 1.9700000e+02 2.0500000e+02]
 [3.1534649e+16 5.6600000e+00 9.0000000e+01 7.4000000e+01]]

Y values: 
[ 376.  267. 1114. ... 1143. 1573. 3052.]



## Split training and test sets

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.1,
    random_state=6
)

## Linear Regression

In [ ]:
from sklearn import linear_model

linReg = linear_model.LinearRegression()
linReg.fit(x_train, y_train)
score = linReg.score(x_test, y_test)
print(f"{score}")

## Random Forest

In [6]:
from sklearn.ensemble import RandomForestRegressor

ranForReg = RandomForestRegressor(max_depth=3, n_estimators=100, random_state=0)
ranForReg.fit(x_train, y_train)
score = ranForReg.score(x_test, y_test)
print(f"{score}")

print(f"Prediction (3452000000000, 2.10, 141, 234): {ranForReg.predict([[3452000000000, 2.10, 141, 234]])} | Actual: 720.0")

### Finding best max depth for random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

maxScore = 0
maxDepth = 1
maxEstimators = 1

for x in range(2, 14):
  for y in range(1, 60):
    ranForRegOpt = RandomForestRegressor(max_depth=x, n_estimators=y, random_state=0)
    ranForRegOpt.fit(x_train, y_train)
    score = ranForRegOpt.score(x_test, y_test)
    if score > maxScore:
      maxScore = score
      maxDepth = x
      maxEstimators = y

print(f"{maxDepth}: {maxScore}")

print(f"Prediction (3452000000000, 2.10, 141, 234): {ranForRegOpt.predict([[3452000000000, 2.10, 141, 234]])} | Actual: 720.0")